In [1]:
import pysr

[juliapkg] Found dependencies: /home/alang/anaconda3/envs/part_gp/lib/python3.10/site-packages/juliapkg/juliapkg.json
[juliapkg] Found dependencies: /home/alang/anaconda3/envs/part_gp/lib/python3.10/site-packages/pysr/juliapkg.json
[juliapkg] Found dependencies: /home/alang/anaconda3/envs/part_gp/lib/python3.10/site-packages/awkward/juliapkg.json
[juliapkg] Found dependencies: /home/alang/anaconda3/envs/part_gp/lib/python3.10/site-packages/juliacall/juliapkg.json
[juliapkg] Locating Julia ^1.10.3
[juliapkg] Querying Julia versions from https://julialang-s3.julialang.org/bin/versions.json
[juliapkg] WARNING: About to install Julia 1.12.2 to /home/alang/anaconda3/envs/part_gp/julia_env/pyjuliapkg/install.
[juliapkg]   If you use juliapkg in more than one environment, you are likely to
[juliapkg]   have Julia installed in multiple locations. It is recommended to
[juliapkg]   install JuliaUp (https://github.com/JuliaLang/juliaup) or Julia
[juliapkg]   (https://julialang.org/downloads) your

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ADTypes ────────────────── v1.20.0
   Installed PrecompileTools ────────── v1.3.3
   Installed StatsAPI ───────────────── v1.8.0
   Installed LineSearches ───────────── v7.5.1
   Installed ForwardDiff ────────────── v1.3.0
   Installed StatsBase ──────────────── v0.34.9
   Installed Optim ──────────────────── v1.13.3
   Installed DifferentiationInterface ─ v0.7.12
    Updating `~/anaconda3/envs/part_gp/julia_env/Project.toml`
  [7d259134] + AwkwardArray v0.1.6
⌅ [6099a3de] + PythonCall v0.9.26
⌅ [8254be44] + SymbolicRegression v1.11.3
  [9e88b42a] ~ Serialization ⇒ v1.11.0
  [458c3c95] ~ OpenSSL_jll ⇒ v3.5.4+0
    Updating `~/anaconda3/envs/part_gp/julia_env/Manifest.toml`
  [47edcb42] + ADTypes v1.20.0
  [79e6a3ab] + Adapt v4.4.0
  [66dad0bd] + AliasTables v1.1.3
  [4fba245c] + ArrayInterface v7.22.0
  [7d259134] + AwkwardArray v0.1.6
  [d360d2e6] + ChainRulesCore v1.26.0
  [bbf7d

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
import os
import sys
workdir = os.getenv("PATH_TO_PARTGP")
sys.path.append(workdir)

import torch
import uproot
import awkward as ak
import numpy
import matplotlib.pyplot as plt
import vector
from pysr import PySRRegressor
import pandas as pd

from weaver.utils.dataset import SimpleIterDataset
from utils.nn_utils.part_prediction import test_load

In [3]:
path_to_outputs = workdir + '/outputs/part_outputs/model_preds/demo_outputs.root'
path_to_config = workdir + '/data_config/JetClass/JetClass_full.yaml'

In [4]:
rootdir = uproot.open(path_to_outputs)

In [5]:
preds = rootdir['predictions']
houtputs = rootdir['hook_outputs']

In [6]:
houtputs['logits'].array()

<Array [[1.29, 8.61, -1.91, ..., -3.96, -5.02], ...] type='5000 * 10 * float32'>

In [7]:
preds.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
scores               | float[10]                | AsDtype("('>f4', (10,))")
labels__label_       | int64_t                  | AsDtype('>i8')
jet_pt               | float                    | AsDtype('>f4')
jet_eta              | float                    | AsDtype('>f4')
jet_phi              | float                    | AsDtype('>f4')
jet_energy           | float                    | AsDtype('>f4')
jet_nparticles       | float                    | AsDtype('>f4')
jet_sdmass           | float                    | AsDtype('>f4')
jet_tau1             | float                    | AsDtype('>f4')
jet_tau2             | float                    | AsDtype('>f4')
jet_tau3             | float                    | AsDtype('>f4')
jet_tau4             | float                    | AsDtype('>f4')


In [8]:
def calc_mass(pt, phi, eta, e):
    """
    Calculate Invariant Mass
    """
    invariant_mass = vector.zip({'pt': pt, 'phi': phi, 'eta': eta, 'E': e}).M
    return invariant_mass

In [9]:
'''
We will be using observers for our inputs for a toy example.
'''

filter_out = ['scores', 'labels__label_']
features = [key for key in preds.keys() if key not in filter_out]

In [10]:
sample_data = []

for data in preds.iterate(features, library = 'ak'):

    data['mass'] = calc_mass(data['jet_pt'], data['jet_phi'], data['jet_eta'], data['jet_energy'])
    sample_data.append(data)

all_data = ak.concatenate(sample_data)

In [11]:
df = pd.DataFrame.from_records(all_data.to_list())

inputs = df.values
input_names = df.columns.tolist()
outputs = houtputs['logits'].array(library = 'np')

In [19]:
sr_outputs = workdir + '/outputs/pysr_outputs/demo_run'

In [ ]:
model = PySRRegressor(
    maxsize=40,
    niterations=50,
    populations=31,
    population_size = 20,
    ncycles_per_iteration = 100,
    binary_operators=["+", "-", "*", "/", "^"],
    unary_operators = ["abs", "log", "sqrt", "relu"],
    constraints = {'^': (-1, 1)},
    output_directory = sr_outputs
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

In [13]:
model.fit(inputs, outputs, variable_names = input_names)

NameError: name 'model' is not defined

In [20]:
import glob

file_list = glob.glob(sr_outputs + '/*.csv')

In [21]:
file_list

['/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output1.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output3.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output7.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output5.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output6.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output4.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output2.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output9.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output8.csv',
 '/home/alang/ppCodes/part_gp/outputs/pysr_outputs/demo_run/hall_of_fame_output10.csv']

In [22]:
test = pd.read_csv(file_list[0])

In [23]:
test

,Complexity,Loss,Equation
0,1,5.572821,1.0721691
1,3,3.811157,0.14541796 / jet_tau1
2,5,3.666763,(jet_tau1 + 0.90555865) ^ -27.097658
3,6,3.636737,sqrt((jet_tau1 + 0.8972418) ^ -50.046387)
4,7,3.525051,((jet_tau1 + 0.90555865) ^ -27.097658) + 0.379...
5,8,3.417118,abs(((jet_tau1 + 0.87090147) ^ -19.497503) + -...
6,10,3.302796,abs((mass * 0.006127323) - ((jet_tau1 + 0.8911...
7,11,3.286971,relu(abs((jet_sdmass * 0.0069556385) - ((jet_t...
8,12,3.285430,abs(((jet_tau1 + 0.89175373) ^ -23.560316) - (...
9,14,3.284775,abs((((jet_sdmass + -0.8128007) * 0.0069556385...


In [24]:
test['Equation'][11]

'abs(((jet_tau1 + 0.89175373) ^ -23.560316) - abs(((jet_sdmass + -1.0216665) - (sqrt(mass) + 1.8461438)) * 0.0069556385))'